<a href="https://colab.research.google.com/github/louisisaacdiouf/Churn-Prediction-with-Artificial-Neural-Network/blob/main/churn_prediction_with_artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEP LEARNING DE A à Z - UDEMY

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import accuracy_score, confusion_matrix
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
import multiprocessing

In [4]:
data = pd.read_csv('/content/drive/MyDrive/__udemy-datasets/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
print("{0:30} {1}".format('Colonnes','Valeurs uniques'))
for col in data.columns:
    print("{0:30} {1}".format(col,len(np.unique(data[col]))))

Colonnes                       Valeurs uniques
RowNumber                      10000
CustomerId                     10000
Surname                        2932
CreditScore                    460
Geography                      3
Gender                         2
Age                            70
Tenure                         11
Balance                        6382
NumOfProducts                  4
HasCrCard                      2
IsActiveMember                 2
EstimatedSalary                9999
Exited                         2


In [6]:
print(np.unique(np.unique(data['Surname'], return_counts=True)[1]))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 28 29 32]


In [7]:
n_cpus = multiprocessing.cpu_count()
n_cpus

2

In [8]:
class ANN:
    def __init__(self, df, n_neuron_in=6, n_neuron_layers=[32,32], n_neuron_out=1, epochs=2, batch_size=50, validation_split=0.3):
        self.df = df
        self.n_neuron_in = n_neuron_in
        self.n_neuron_layers = n_neuron_layers
        self.n_neuron_out = n_neuron_out
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        
    def train_preprocessing(self, X):
        # Encodeur
        cat_encoder = OneHotEncoder()

        # Encodage des variables catégorielles
        x_categorical = cat_encoder.fit_transform(X.select_dtypes('object')).toarray()
        self.cat_encoder = cat_encoder

        # Nombre de colonnes avant encodage
        n_column_ini = len(X.columns)

        # Suppression des colonnes à encoder
        X = X.drop(columns=X.select_dtypes('object').columns)

        # Assemblage du nouveau tableau
        X = np.concatenate((x_categorical,np.array(X)), axis=1)
        n_vars = len(X[0])
        
        # Scaleur
        scaler = StandardScaler()

        # Normalisation des données
        X = scaler.fit_transform(X)
        self.scaler = scaler
        return X
        
       
        
    def training(self):
        df = self.df
        # ---------------------------------------------------
        # ------------------ PREPROCESSING ------------------
        # ---------------------------------------------------
        # Variables inutiles
        
        df = data.drop(columns=['RowNumber','CustomerId','Surname'])
        
        # Création des variables
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        
        self.X = X
        
        # Preprocessing
        X = self.train_preprocessing(X)
        
        n_vars = len(X[0])
        
        # Création des jeux de données train et test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        
        # --------------------------------------------------
        # --------------- CREATION DU MODELE ---------------
        # --------------------------------------------------  
        
        n_neuron_in = self.n_neuron_in
        n_neuron_layers = self.n_neuron_layers
        n_neuron_out = self.n_neuron_out
        epochs = self.epochs
        batch_size = self.batch_size
        validation_split = self.validation_split

        model = Sequential()
        model.add(Dense(units=n_neuron_in,
                        activation='relu',
                        # kernel_initializer='uniform',
                        input_dim=n_vars))

        for n in n_neuron_layers:
            model.add(Dense(units=n, activation='relu',
                        # kernel_initializer='uniform'
                        ))

        model.add(Dense(units=n_neuron_out, activation='sigmoid',
                        # kernel_initializer='uniform'
                        ))

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, use_multiprocessing=True)
        
        y_pred = (model.predict(X_test) > 0.8).astype(int)

        self.training_confusion_matrix = confusion_matrix(y_test,y_pred)

        self.training_evaluation = model.evaluate(X_test, y_test, verbose=0)
        
        self.model = model
        
        
    def kFold(self):
        df = self.df
        # ---------------------------------------------------
        # ------------------ PREPROCESSING ------------------
        # ---------------------------------------------------
        # Variables inutiles
        
        df = data.drop(columns=['RowNumber','CustomerId','Surname'])
        
        # Création des variables
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        
        self.X = X
        
        # Preprocessing
        X = self.train_preprocessing(X)
        
        n_vars = len(X[0])
        
        # Création des jeux de données train et test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        
        # --------------------------------------------------
        # --------------- CREATION DU MODELE ---------------
        # --------------------------------------------------  
        
        n_neuron_in = self.n_neuron_in
        n_neuron_layers = self.n_neuron_layers
        n_neuron_out = self.n_neuron_out
        epochs = self.epochs
        batch_size = self.batch_size
        validation_split = self.validation_split
        
        def build_classifier():
            model = Sequential()
            model.add(Dense(units=n_neuron_in,
                            activation='relu',
                            # kernel_initializer='uniform',
                            input_dim=n_vars))
            model.add(Dropout(rate=0.1))
            
            for n in n_neuron_layers:
                model.add(Dense(units=n, activation='relu',
                            # kernel_initializer='uniform'
                            ))
                model.add(Dropout(rate=0.1))

            model.add(Dense(units=n_neuron_out, activation='sigmoid',
                            # kernel_initializer='uniform'
                            ))

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            
            return model
        
        classifier = KerasClassifier(model=build_classifier, batch_size=self.batch_size, epochs=self.epochs)

        # n_cpus = multiprocessing.cpu_count()
        precisions = cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=10)
        print(f"Precision mean : {precisions.mean()}")
        print(f"Precision STD : {precisions.std()}")
        
        classifier.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, use_multiprocessing=True)
              
        y_pred = (classifier.predict(X_test) > 0.8).astype(int)

        self.kFold_confusion_matrix = confusion_matrix(y_test,y_pred)

        self.kFold_score = classifier.score(X_test, y_test)
        
        self.kFoldClassifier = classifier


    def GridSearch(self, bs, ep, optim, cross_val):
        df = self.df
        # ---------------------------------------------------
        # ------------------ PREPROCESSING ------------------
        # ---------------------------------------------------
        # Variables inutiles
        
        df = data.drop(columns=['RowNumber','CustomerId','Surname'])
        
        # Création des variables
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        
        self.X = X
        
        # Preprocessing
        X = self.train_preprocessing(X)
        
        n_vars = len(X[0])
        
        # Création des jeux de données train et test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        
        # --------------------------------------------------
        # --------------- CREATION DU MODELE ---------------
        # --------------------------------------------------  
        
        n_neuron_in = self.n_neuron_in
        n_neuron_layers = self.n_neuron_layers
        n_neuron_out = self.n_neuron_out
        epochs = self.epochs
        batch_size = self.batch_size
        validation_split = self.validation_split
        
        def build_classifier():
            model = Sequential()
            model.add(Dense(units=n_neuron_in,
                            activation='relu',
                            # kernel_initializer='uniform',
                            input_dim=n_vars))
            model.add(Dropout(rate=0.1))
            
            for n in n_neuron_layers:
                model.add(Dense(units=n, activation='relu',
                            # kernel_initializer='uniform'
                            ))
                model.add(Dropout(rate=0.1))

            model.add(Dense(units=n_neuron_out, activation='sigmoid',
                            # kernel_initializer='uniform'
                            ))

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            
            return model
        
        classifier = KerasClassifier(model=build_classifier, batch_size=self.batch_size, epochs=self.epochs)
        parameters = {
            "batch_size": bs,
            "epochs": ep,
            "optimizer": optim,
        }

        # n_cpus = multiprocessing.cpu_count()
        gridsearch = GridSearchCV(estimator=classifier,
                                  param_grid=parameters,
                                  scoring="accuracy",
                                  cv=cross_val)
        
        gridsearch.fit(X_train, y_train, use_multiprocessing=True)
        
        y_pred = (gridsearch.predict(X_test) > 0.8).astype(int)
        self.gridsearch_confusion_matrix = confusion_matrix(y_test,y_pred)
        
        self.gridsearch = gridsearch
        
        
    def predict_preprocessing(self,X):
        x_categorical = self.cat_encoder.transform(X.select_dtypes('object')).toarray()
        X = X.drop(columns=X.select_dtypes('object').columns)
        X = np.concatenate((x_categorical,np.array(X)), axis=1)        
        X = self.scaler.transform(X)
        return X        
        
    def predict(self,X,strategy='classic'):
        X = self.predict_preprocessing(X)
        if strategy=='classic':
            return self.model.predict(X)
        elif strategy=='kFold':
            return self.kFoldClassifier.predict_proba(X)
        elif strategy=='gridSearch':
            return self.gridsearch.predict_proba(X)

In [9]:
ai = ANN(data, epochs=150, n_neuron_layers=[16,8,4,8,16], batch_size=128)

In [10]:
cols = list(data.drop(columns=['RowNumber','CustomerId','Surname']).columns[:-1])
client = pd.DataFrame(data=[[619,'France','Male',40,3,60000,2,1,1,50000]], columns=cols)
# print(client.dtypes)
client

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Male,40,3,60000,2,1,1,50000


In [11]:
ai.training()

Epoch 1/150
39/39 [==============================] - 1s 8ms/step - loss: 0.6478 - accuracy: 0.7969 - val_loss: 0.5964 - val_accuracy: 0.7995
Epoch 2/150
39/39 [==============================] - 0s 3ms/step - loss: 0.5461 - accuracy: 0.7969 - val_loss: 0.4998 - val_accuracy: 0.7995
Epoch 3/150
39/39 [==============================] - 0s 5ms/step - loss: 0.4965 - accuracy: 0.7969 - val_loss: 0.4845 - val_accuracy: 0.7995
Epoch 4/150
39/39 [==============================] - 0s 4ms/step - loss: 0.4834 - accuracy: 0.7969 - val_loss: 0.4764 - val_accuracy: 0.7995
Epoch 5/150
39/39 [==============================] - 0s 3ms/step - loss: 0.4743 - accuracy: 0.7969 - val_loss: 0.4692 - val_accuracy: 0.7995
Epoch 6/150
39/39 [==============================] - 0s 4ms/step - loss: 0.4661 - accuracy: 0.7969 - val_loss: 0.4627 - val_accuracy: 0.7995
Epoch 7/150
39/39 [==============================] - 0s 3ms/step - loss: 0.4584 - accuracy: 0.7969 - val_loss: 0.4561 - val_accuracy: 0.7995
Epoch 8/150
3

In [12]:
print(f"training loss = {ai.training_evaluation[0]} \ntraining accuracy = {ai.training_evaluation[1]}")
print("\n---------------------------------------------------------")
print("----------------- CONFUSION MATRIX ----------------------")
print("---------------------------------------------------------")
print(ai.training_confusion_matrix)
print(f"\nProbability of churn = {ai.predict(client)[0][0] * 100} %")

training loss = 0.40822774171829224 
training accuracy = 0.8323333263397217

---------------------------------------------------------
----------------- CONFUSION MATRIX ----------------------
---------------------------------------------------------
[[2365   14]
 [ 544   77]]

Probability of churn = 6.005248427391052 %


In [13]:
ai.kFold()

Epoch 1/150
50/50 [==============================] - 1s 2ms/step - loss: 0.5916 - accuracy: 0.7957
Epoch 2/150
50/50 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 0.7978
Epoch 3/150
50/50 [==============================] - 0s 2ms/step - loss: 0.5041 - accuracy: 0.7978
Epoch 4/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4899 - accuracy: 0.7978
Epoch 5/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4789 - accuracy: 0.7978
Epoch 6/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4765 - accuracy: 0.7978
Epoch 7/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4693 - accuracy: 0.7978
Epoch 8/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4693 - accuracy: 0.7978
Epoch 9/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.7978
Epoch 10/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.7978
Epoch 11/

In [14]:
print(f"kFold score = {ai.kFold_score}")
print("\n---------------------------------------------------------")
print("----------------- CONFUSION MATRIX ----------------------")
print("---------------------------------------------------------")
print(ai.kFold_confusion_matrix)
print(f"\nkFold Probability of churn = {ai.predict(client,strategy='kFold')[1] * 100} %")

kFold score = 0.8493333333333334

---------------------------------------------------------
----------------- CONFUSION MATRIX ----------------------
---------------------------------------------------------
[[2234  145]
 [ 307  314]]
1/1 [==============================] - 0s 21ms/step

kFold Probability of churn = 13.285768032073975 %


In [15]:
# ai.GridSearch([20,32,40], [500], ["adam"], cross_val=5)
ai.GridSearch([32,64,80], [100,200], ["adam","rmsprop"], cross_val=5)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
70/70 [==============================] - 0s 3ms/step - loss: 0.4570 - accuracy: 0.8057
Epoch 9/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4507 - accuracy: 0.8116
Epoch 10/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4465 - accuracy: 0.8086
Epoch 11/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4423 - accuracy: 0.8091
Epoch 12/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4384 - accuracy: 0.8121
Epoch 13/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4363 - accuracy: 0.8087
Epoch 14/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4341 - accuracy: 0.8145
Epoch 15/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4235 - accuracy: 0.8191
Epoch 16/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4218 - accuracy: 0.8159
Epoch 17/100
70/70 [================

In [16]:
print("Best Parameters :")
print(ai.gridsearch.best_params_)
print("\n-----------------------------------\n")
print("Best Estimator :")
print(ai.gridsearch.best_estimator_)
print("\n-----------------------------------\n")
print("Best Score :")
print(ai.gridsearch.best_score_)
print("\n---------------------------------------------------------")
print("----------------- CONFUSION MATRIX ----------------------")
print("---------------------------------------------------------")
print(ai.gridsearch_confusion_matrix)

print(f"\nGridSearch Probability of churn = {ai.predict(client,strategy='gridSearch')[1] * 100} %")

Best Parameters :
{'batch_size': 80, 'epochs': 200, 'optimizer': 'rmsprop'}

-----------------------------------

Best Estimator :
KerasClassifier(
	model=<function ANN.GridSearch.<locals>.build_classifier at 0x7ff316cf7320>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=80
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	class_weight=None
)

-----------------------------------

Best Score :
0.8562857142857142

---------------------------------------------------------
----------------- CONFUSION MATRIX ----------------------
---------------------------------------------------------
[[2252  127]
 [ 307  314]]
1/1 [==============================] - 0s 22ms/step

GridSearch Probability of churn = 9.886682033538818 %


### Réponse : Le client étudié a de fortes chances de rester avec la banque